In [1]:
import pandas as pd
from __future__ import division
import csv
from datetime import datetime
from collections import Counter

In [2]:
import matplotlib.pyplot as plt

import stuff

In [ ]:
data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160213.txt')

In [3]:
list_data =[]
list_weeks = ['160102', '160109', '160116','160130','160206','160213',
             '160220','160227','160305','160312','160319','160326','160402']
# list_weeks = ['151205','150530']

for u in list_weeks:
    url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_' + u + '.txt'
    print(url)
    list_data.append(pd.read_csv(url))

data = pd.concat(list_data)

http://web.mta.info/developers/data/nyct/turnstile/turnstile_160102.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160109.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160116.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160130.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160206.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160213.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160220.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160227.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160305.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160312.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160319.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160326.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt


read data in + merge data

In [4]:
data.columns = data.columns.str.replace(' ','')

remove spaces from exits column

In [5]:
data["DATETIME"] = data["DATE"] + ' ' + data["TIME"]

In [6]:
data['DATETIME'] = pd.to_datetime(data.DATETIME, format='%m/%d/%Y %H:%M:%S')

In [ ]:
data.head()

create datetime and set format

In [7]:
data['HOUR'] = data.DATETIME.apply(lambda x: x.hour)

In [ ]:
#data['WEEKDAY']=data.DATETIME.apply(lambda x: 'Weekday' if 0<=x.weekday()<5 else 'Weekend')

#same as above in 2 steps
#weekdaycheck = lambda x: 'Weekday' if 0<=x.weekday()<5 else 'Weekend'
#data['WEEKDAY']=data.DATETIME.apply(weekdaycheck)


In [8]:
daysofweek = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday', 'Monday']

#whatdayofweek = lambda x: daysofweek[x.weekday()]

def whatdayofweek(x):
    if x.hour>21:
        return daysofweek[x.weekday()+1]
    else:
        return daysofweek[x.weekday()]

data['DAYOFWEEK'] = data.DATETIME.apply(whatdayofweek)



# for r in range(data.shape[0]):
#     if data.ix[r,'DAYOFWEEK'] == :
#         data.ix[r,'dTOTALACTUAL'] = groupedweekdayend.ix[r, 'dTOTAL']/(5*13)
#     else:
#         groupedweekdayend.ix[r,'dTOTALACTUAL'] = groupedweekdayend.ix[r, 'dTOTAL']/(2*13)
# #data['DAYOFWEEKVALUE'] = data.DATETIME.apply(dayofweekvalue)

In [9]:
def weekendorday(x):
    if x.hour>21:
        if x.weekday()+1<5 or (x.weekday()+1)==7 :
            return 'Weekday'
        else:
            return 'Weekend'
    else:
        if x.weekday()<5:
            return 'Weekday'
        else:
            return 'Weekend'
        
data['WEEKDAY']=data.DATETIME.apply(weekendorday)

In [ ]:
data.head(100)

create day of week by applying function to DATETIME

In [10]:
data['dENTRIES'] = data.ENTRIES.shift(-1) - data.ENTRIES
data['dEXITS'] = data.EXITS.shift(-1) - data.EXITS

###for considering deltas as time(i+1)-time(i)

create deltas by shifting from one above

In [ ]:
#data['dENTRIES'] = data.ENTRIES - data.ENTRIES.shift(1)
#data['dEXITS'] = data.EXITS - data.EXITS.shift(1)
#data['dTOTAL'] = data.dENTRIES+ data.dEXITS

###this is for considering deltas as time(i)-time(i-1)

In [ ]:
data.columns

In [11]:
data.dENTRIES.fillna(0, inplace = True)

In [12]:
data.dEXITS.fillna(0, inplace = True)

In [22]:
data['dTOTAL'] = data.dENTRIES+ data.dEXITS


In [ ]:
data[data.dTOTAL.isnull()].shape

In [23]:
datatest = data[(data.HOUR ==0)|(data.HOUR ==1)|(data.HOUR==22)|(data.HOUR==23)]
# same as mask

In [ ]:
datatest.shape

filter based on time we care about

In [43]:
datatest2 = datatest[(datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)]
# mask2 = (datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)


In [25]:
datatest2.shape

(381110, 18)

filter a second time based on change in deltas to account for switching machine/station etc.  
Lose out on 411197 - 407288 = 3909 total entries  
3909 is 0.95% of our data that we lose

In [44]:
weekdayend = datatest2.groupby([datatest2.WEEKDAY, datatest2.STATION]).sum()
#final_weekday_or_end = datatest2.groupby([datatest2.STATION]).sum()

In [45]:
groupedweekdayend = (weekdayend
    .reset_index()
    .sort_values(['WEEKDAY','dTOTAL'], ascending=(True,False))
)

In [46]:
for r in range(groupedweekdayend.shape[0]):
    if groupedweekdayend.ix[r,'WEEKDAY'] =='Weekday':
        groupedweekdayend.ix[r,'dTOTALACTUAL'] = groupedweekdayend.ix[r, 'dTOTAL']/(5*13)
    else:
        groupedweekdayend.ix[r,'dTOTALACTUAL'] = groupedweekdayend.ix[r, 'dTOTAL']/(2*13)

In [47]:
groupedweekdayend

,WEEKDAY,STATION,ENTRIES,EXITS,HOUR,dENTRIES,dEXITS,dTOTAL,dTOTALACTUAL
348,Weekday,TIMES SQ-42 ST,248297993188,200061728110,56350,301336,94767,396103,6093.892308
61,Weekday,34 ST-PENN STA,283199806601,135403657282,98370,242114,150509,392623,6040.353846
68,Weekday,42 ST-PORT AUTH,248464102912,171570532622,39460,204249,117019,321268,4942.584615
59,Weekday,34 ST-HERALD SQ,88061961518,69417895750,60927,199947,82820,282767,4350.261538
9,Weekday,125 ST,42696477765,34123836158,24410,104701,132874,237575,3655.000000
229,Weekday,GRD CNTRL-42 ST,272324104096,96017326785,30633,159298,76394,235692,3626.030769
253,Weekday,JKSN HT-ROOSVLT,8167632094,6087019758,25162,66619,164060,230679,3548.907692
109,Weekday,86 ST,91169547144,41536111647,40490,123065,101102,224167,3448.723077
46,Weekday,23 ST,267718211941,233544879122,44165,163507,52367,215874,3321.138462
86,Weekday,59 ST COLUMBUS,230205626112,204127310176,46736,168131,44034,212165,3264.076923


In [48]:
testweekend = groupedweekdayend[(groupedweekdayend.WEEKDAY == 'Weekend')]

In [60]:
testweekend.head(10)

,WEEKDAY,STATION,ENTRIES,EXITS,HOUR,dENTRIES,dEXITS,dTOTAL,dTOTALACTUAL
721,Weekend,TIMES SQ-42 ST,57329716411,46171042394,12397,116772,38472,155244,5970.923077
387,Weekend,14 ST-UNION SQ,34631837602,13003137167,152,114553,39337,153890,5918.846154
441,Weekend,42 ST-PORT AUTH,61736254246,41678094932,8359,86569,39677,126246,4855.615385
434,Weekend,34 ST-PENN STA,82391163806,42217688221,21370,74645,50639,125284,4818.615385
731,Weekend,W 4 ST-WASH SQ,20895976200,23300734723,76,85393,27217,112610,4331.153846
432,Weekend,34 ST-HERALD SQ,20804003043,16485747670,13409,65899,32853,98752,3798.153846
382,Weekend,125 ST,11270630524,9023798758,5393,43334,51654,94988,3653.384615
482,Weekend,86 ST,22155410923,10825844356,8919,46841,44418,91259,3509.961538
419,Weekend,23 ST,84436307464,72571479888,9807,62077,27320,89397,3438.346154
386,Weekend,14 ST,11763536540,3487360718,8136,58210,26617,84827,3262.576923


In [32]:
dayofweek = datatest2.groupby([datatest2.DAYOFWEEK, datatest2.STATION]).sum()

In [33]:
groupeddayofweek = (dayofweek
    .reset_index()
    .sort_values(['DAYOFWEEK','dTOTAL'], ascending=(False,False))
)

In [34]:
groupeddayofweek.head()

,DAYOFWEEK,STATION,ENTRIES,EXITS,HOUR,dENTRIES,dEXITS,dTOTAL
2586,Wednesday,TIMES SQ-42 ST,49683111953,40012443840,11270,60920,17060,77980
2299,Wednesday,34 ST-PENN STA,56626465995,27100659196,19766,47142,28650,75792
2306,Wednesday,42 ST-PORT AUTH,49105866259,33860913384,7881,36869,22111,58980
2297,Wednesday,34 ST-HERALD SQ,17612431381,13883608150,12190,39924,15153,55077
2467,Wednesday,GRD CNTRL-42 ST,52966586520,18202048809,6145,34376,13137,47513


In [50]:
whatdayindex = lambda x: daysofweek.index(x)

In [51]:
groupeddayofweek['DAYOFWEEKVALUE'] = groupeddayofweek.DAYOFWEEK.apply(whatdayindex)

In [52]:
groupeddayofweek = groupeddayofweek.sort_values(['DAYOFWEEKVALUE','dTOTAL'], ascending = [True, False])

In [53]:
groupeddayofweek.dTOTAL = groupeddayofweek.dTOTAL/13

In [54]:
testsunday = groupeddayofweek[(groupeddayofweek.DAYOFWEEK == 'Sunday')]

testmonday = groupeddayofweek[(groupeddayofweek.DAYOFWEEK == 'Monday')]

testsaturday = groupeddayofweek[(groupeddayofweek.DAYOFWEEK == 'Saturday')]

testtuesday = groupeddayofweek[(groupeddayofweek.DAYOFWEEK == 'Tuesday')]

testwed = groupeddayofweek[(groupeddayofweek.DAYOFWEEK == 'Wednesday')]

testfriday=groupeddayofweek[(groupeddayofweek.DAYOFWEEK == 'Friday')]

In [55]:
testsunday.head()

,DAYOFWEEK,STATION,ENTRIES,EXITS,HOUR,dENTRIES,dEXITS,dTOTAL,DAYOFWEEKVALUE
1467,Sunday,TIMES SQ-42 ST,49680137782,40009769718,12397,103208,34285,813.568047,6
1180,Sunday,34 ST-PENN STA,56666310725,27094105278,21139,62246,41933,616.443787,6
1187,Sunday,42 ST-PORT AUTH,51250413079,35568018253,8151,61373,30178,541.721893,6
1178,Sunday,34 ST-HERALD SQ,17608764334,13880333476,13409,59116,29743,525.792899,6
1133,Sunday,14 ST-UNION SQ,17316371190,6501976599,76,56627,20718,457.662722,6


In [56]:
testmonday.head()

,DAYOFWEEK,STATION,ENTRIES,EXITS,HOUR,dENTRIES,dEXITS,dTOTAL,DAYOFWEEKVALUE
721,Monday,TIMES SQ-42 ST,49680791690,40010366611,11270,49215,15668,383.923077,0
434,Monday,34 ST-PENN STA,56644082136,27048852578,19559,38823,21543,357.195266,0
441,Monday,42 ST-PORT AUTH,49219671838,33875534492,7883,33299,17743,302.023669,0
432,Monday,34 ST-HERALD SQ,17609422132,13880927958,12190,29010,13163,249.544379,0
382,Monday,125 ST,8339530124,6789719180,4889,18686,21941,240.396450,0


In [57]:
testsaturday.head()

,DAYOFWEEK,STATION,ENTRIES,EXITS,HOUR,dENTRIES,dEXITS,dTOTAL,DAYOFWEEKVALUE
1041,Saturday,NEWARK C,8417344,511395,227,157985,90047,1467.644970,5
806,Saturday,34 ST-HUDSON YD,3079031,3369660,667,83564,117658,1190.662722,5
986,Saturday,HOYT ST,30356790,28642367,230,101185,69716,1011.248521,5
1010,Saturday,LACKAWANNA,11582950,17209889,237,85535,64857,889.893491,5
1085,Saturday,ST. GEORGE,63964105,193734,138,103635,6,613.260355,5


In [58]:
testtuesday.head()

,DAYOFWEEK,STATION,ENTRIES,EXITS,HOUR,dENTRIES,dEXITS,dTOTAL,DAYOFWEEKVALUE
2213,Tuesday,TIMES SQ-42 ST,49681900361,40011358068,11270,52403,16599,408.295858,1
1926,Tuesday,34 ST-PENN STA,56707637834,27074032038,19605,42820,25860,406.390533,1
1933,Tuesday,42 ST-PORT AUTH,49153584055,33859473810,7882,35087,21386,334.159763,1
1924,Tuesday,34 ST-HERALD SQ,17610516198,13881890149,12167,35052,13973,290.088757,1
2118,Tuesday,JKSN HT-ROOSVLT,1635260640,1218888964,5060,13588,30595,261.437870,1


In [59]:
testfriday.head()

,DAYOFWEEK,STATION,ENTRIES,EXITS,HOUR,dENTRIES,dEXITS,dTOTAL,DAYOFWEEKVALUE
61,Friday,34 ST-PENN STA,56613801105,27093073478,19766,62389,42222,619.000000,4
348,Friday,TIMES SQ-42 ST,49685589047,40014700665,11270,71861,26513,582.094675,4
68,Friday,42 ST-PORT AUTH,51675733125,35893043333,7930,57169,31401,524.082840,4
59,Friday,34 ST-HERALD SQ,17615581418,13886444291,12190,52822,23434,451.218935,4
229,Friday,GRD CNTRL-42 ST,52701414249,19396442145,6099,43118,25347,405.118343,4
